In [1]:
from transformers import AutoTokenizer,AutoModel
# from thuglm.modeling_chatglm import ChatGLMForConditionalGeneration
import torch
from peft import get_peft_model, LoraConfig, TaskType


In [ ]:
model = AutoModel.from_pretrained(
    "/root/chatglm6b-local/models--THUDM--chatglm-6b/snapshots/bf0f5cfb575eebebf9b655c5861177acfee03f16", trust_remote_code=True).half().cuda()

peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    inference_mode=False, r=8, lora_alpha=32, lora_dropout=0.1,
    target_modules=['query_key_value',], # 见下ChatGLMModel...
)
model = get_peft_model(model, peft_config)

# 在这里加载lora模型，注意修改chekpoint
peft_path = "/root/zero_nlp/simple_thu_chatglm6b/test004/chatglm-lora.pt"
model.load_state_dict(torch.load(peft_path,map_location='cuda'), strict=False)
model = model.half().cuda() # important!
for name, param in model.named_parameters():
    if param.device != torch.device('cuda:0'):
        print(f"Warning: Parameter {name} is on {param.device}")
model.eval()

Explicitly passing a `revision` is encouraged when loading a configuration with custom code to ensure no malicious code has been contributed in a newer revision.
Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

/root/miniconda3/lib/python3.10/site-packages/peft/tuners/lora.py:173: UserWarning: fan_in_fan_out is set to True but the target module is not a Conv1D. Setting fan_in_fan_out to False.
  warnings.warn(


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): ChatGLMForConditionalGeneration(
      (transformer): ChatGLMModel(
        (word_embeddings): Embedding(130528, 4096)
        (layers): ModuleList(
          (0-27): 28 x GLMBlock(
            (input_layernorm): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
            (attention): SelfAttention(
              (rotary_emb): RotaryEmbedding()
              (query_key_value): MergedLinear(
                in_features=4096, out_features=12288, bias=True
                (lora_dropout): Dropout(p=0.1, inplace=False)
                (lora_A): Linear(in_features=4096, out_features=16, bias=False)
                (lora_B): Conv1d(16, 8192, kernel_size=(1,), stride=(1,), groups=2, bias=False)
              )
              (dense): Linear(in_features=4096, out_features=4096, bias=True)
            )
            (post_attention_layernorm): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
            (mlp): GLU(
        

In [3]:
tokenizer = AutoTokenizer.from_pretrained("/root/chatglm6b-local/models--THUDM--chatglm-6b/snapshots/bf0f5cfb575eebebf9b655c5861177acfee03f16", trust_remote_code=True)

Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.


In [7]:
# text ="为什么冰红茶和柠檬茶的味道一样？"

# with torch.autocast("cuda"):
#     res, history = model.chat(tokenizer=tokenizer, query=text,max_length=300)
#     print(res)

text = "类型#上衣*材质#牛仔布*颜色#白色*风格#简约*图案#刺绣*衣样式#外套*衣款式#破洞"

with torch.no_grad():
    # Convert attention mask to bool type explicitly
    inputs = tokenizer(text, return_tensors="pt").to('cuda')
    print(inputs)
    
    # Generate using the base model's generate method directly
    generate_ids = model.generate(
        **inputs,
        max_new_tokens=100
    )
    
    # Decode output
    output = tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
    print(output)

{'input_ids': tensor([[     5,  65421,     61,  75898,     32,  68554,     61,  77257,  64555,
             32,  65107,     61,  66268,     32,  65347,     61,  71689,     32,
          69768,     61,  85428,     32,  65173,  73942,     61,  70984,     32,
          65173,  70936,     61,  64703,  65509, 130001, 130004]],
       device='cuda:0')}
类型#上衣*材质#牛仔布*颜色#白色*风格#简约*图案#刺绣*衣样式#外套*衣款式#破洞 这款上衣采用了简约的牛仔布材质,白色刺绣图案点缀其中,增添了不少时尚感。衣款式采用了破洞设计,个性十足。
